In [1]:
import astrolights

Initilization module astrolights


In [2]:
import cv2
import os
import re
import numpy as np

In [3]:
#import importlib
#importlib.reload(astrolights)

In [4]:
tmpDir='/mnt/data/Photos/Repository/tmp'
tmpGeneratedFiles = []
regenerateOffset = False
regenerateFlat = False
regenerateDark = False

In [5]:
imagesDirectory = '/mnt/data/Photos/Repository/2018/10-Harz'
categories = astrolights.files.categorifyRawImages(imagesDirectory)

flatDirectory = '/mnt/data/Photos/Repository/2018/10-Harz'
categoriesFlat = astrolights.files.categorifyRawImages(flatDirectory)

darkDirectory = '/mnt/data/Photos/Repository/2018/10-Harz'
categoriesDark = astrolights.files.categorifyRawImages(darkDirectory)

offsetDirectory = '/mnt/data/Photos/Repository/2018/10-Harz'
categoriesOffset = astrolights.files.categorifyRawImages(offsetDirectory)

In [6]:
imageList = categories['Cluster_LIGHT']
flatList = categoriesFlat['FLAT2']
darkList = categoriesDark['DARK']
offsetList = categoriesOffset['BIAS']


In [7]:
categories

{'BIAS': ['/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7U.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT77.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7M.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7V.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7L.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT73.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7P.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT74.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT75.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7K.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7W.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7R.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT79.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7B.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT7H.CR2',
  '/mnt/data/Photos/Repository/2018/10-Harz/JPB

# Generating master offset

The generation of the master offset is just a median of all offset image

In [8]:
if regenerateOffset:
    masterOffset = astrolights.processImages.calculateMedianFromFiles(offsetList)
    masterOffset = masterOffset.astype(np.uint16, copy=False)
    cv2.imwrite(tmpDir + '/' + "masterOffset.pgm", masterOffset)
else:
    try:
        masterOffset = cv2.imread(tmpDir + '/' + "masterOffset.pgm", flags=cv2.IMREAD_GRAYSCALE | cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH)
    except:
        print('File: {} for the Master Offset not found.'.format(tmpDir + '/' + "masterOffset.pgm"))

In [9]:
masterOffset

array([[134,   0,  53, ...,  21,   0,   0],
       [ 32,  15,   0, ...,  52,  37, 142],
       [ 67,   0,   0, ...,   0,   0,  48],
       ..., 
       [  0,  48,   0, ...,   0,  40,   0],
       [  0,   0, 107, ...,  75,  75,  37],
       [  0,  42,   0, ...,  91,   0,   0]], dtype=uint16)

# Generating master Dark

In [10]:
if regenerateDark:
    masterDark = astrolights.processImages.calculateMedianFromFiles(darkList)
    masterDark = masterDark.astype(np.uint16, copy=False)
    cv2.imwrite(tmpDir + '/' + "masterDark.pgm", masterDark)
else:
    try:
        masterDark = cv2.imread(tmpDir + '/' + "masterDark.pgm", flags=cv2.IMREAD_GRAYSCALE | cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH)
    except:
        print('File: {} for the Master Dark not found.'.format(tmpDir + '/' + "masterDark.pgm"))

In [11]:
masterDark

array([[ 550,    0,    0, ...,    0,    0,    0],
       [ 123,    0,    0, ...,    0,    0,  188],
       [2536,    0,  415, ...,    0,    0, 1921],
       ..., 
       [   0,    0,    0, ...,    0,    0,   48],
       [   0,    0,    0, ...,   97,   96,    0],
       [ 724,    0,    0, ...,    0,   40,   80]], dtype=uint16)

# Generating master Flat

In [12]:
if regenerateFlat:
    masterFlat = astrolights.processImages.calculateMedianFromFiles(flatList)
    masterFlat = masterFlat.astype(np.uint16, copy=False)
    cv2.imwrite(tmpDir + '/' + "masterFlat.pgm", masterFlat)
else:
    try:
        masterFlat = cv2.imread(tmpDir + '/' + "masterFlat.pgm", flags=cv2.IMREAD_GRAYSCALE | cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH)
    except:
        print('File: {} for the Master Flat not found.'.format(tmpDir + '/' + "masterFlat.pgm"))

In [13]:
meanFlat = np.mean(masterFlat)

In [14]:
masterFlat

array([[24928, 14521, 22772, ..., 16663, 28292, 16364],
       [14546, 12090, 14037, ..., 13495, 17081, 13878],
       [23201, 14117, 23227, ..., 16401, 27664, 17049],
       ..., 
       [24097, 15023, 24526, ..., 17378, 28586, 17548],
       [15382, 13081, 14895, ..., 15049, 18100, 14982],
       [24796, 15136, 23817, ..., 16872, 28292, 17796]], dtype=uint16)

In [15]:
meanFlat

23736.1620684796

# Calculate images

In [16]:
#correctedImages = astrolights.processImages.correctImages(imageList, masterOffset, masterDark, masterFlat)

In [17]:
i=0
for image in imageList:
    correctedImage = astrolights.processImages.correctImages(image, masterOffset, masterDark, masterFlat)
    #correctedImage = correctedImage/correctedImage.max()*65535
    
    # Correction Gamma
    #print('Ajust Gamma')
    correctedImage = correctedImage.astype(np.float64, copy=False) 
    print(correctedImage.min())
    print(correctedImage.max())
    
    correctedImage -= correctedImage.min()
    correctedImage = (correctedImage / correctedImage.max()) * 65535
    print(correctedImage.min())
    print(correctedImage.max())
    correctedImage = correctedImage.astype(np.uint16, copy=False)
    print(correctedImage.shape)
    BGR_image = cv2.cvtColor(correctedImage, cv2.COLOR_BayerBG2BGR_EA)

    bitDepth = 2.0**16
    BGR_image = np.power(BGR_image/bitDepth, 1/2.1)*bitDepth
    
    #Correction of the white balance
    #print('Ajust White balance')
    #BGR_image = astrolights.histogram.ajustWhiteBalance(BGR_image)
    
    #Normalization of the histogram
    print('Normalize the histogram')
    BGR_image = astrolights.histogram.normalizeHistogram(BGR_image, sigma1=3, sigma2=10)
    
    BGR_image = BGR_image.astype(np.uint16, copy=False)
    i += 1
    print("Writing: Corrected_{:04d}.png".format(i))
    cv2.imwrite(tmpDir + '/' + "Corrected_{:04d}.png".format(i), BGR_image)


dcraw -d -4 -t 0 /mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT9Y.CR2
-47719.9973823
121860.507729
0.0
65535.0
(3465, 5202)
Normalize the histogram
Writing: Corrected_0001.png
dcraw -d -4 -t 0 /mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT9K.CR2
-30466.8792079
123819.898206
0.0
65535.0
(3465, 5202)
Normalize the histogram
Writing: Corrected_0002.png
dcraw -d -4 -t 0 /mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT95.CR2
-47851.1185122
122493.848426
0.0
65535.0
(3465, 5202)
Normalize the histogram
Writing: Corrected_0003.png
dcraw -d -4 -t 0 /mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT9H.CR2
-29176.117925
123681.077419
0.0
65535.0
(3465, 5202)
Normalize the histogram
Writing: Corrected_0004.png
dcraw -d -4 -t 0 /mnt/data/Photos/Repository/2018/10-Harz/JPB-8F7_NT9V.CR2
-46916.2734195
123682.068948
0.0
65535.0
(3465, 5202)
Normalize the histogram
Writing: Corrected_0005.png


# Aligns images

Hugin is used to align the generated images:

    align_image_stack -v --gpu -a Aligned_ Corrected_*.png

# Add aligned Images

In [18]:
alignedImages = astrolights.files.listAllFileinDir(tmpDir, '^Aligned_.*')

In [19]:
alignedImages

['/mnt/data/Photos/Repository/tmp/Aligned_0003.tif',
 '/mnt/data/Photos/Repository/tmp/Aligned_0001.tif',
 '/mnt/data/Photos/Repository/tmp/Aligned_0004.tif',
 '/mnt/data/Photos/Repository/tmp/Aligned_0002.tif',
 '/mnt/data/Photos/Repository/tmp/Aligned_0000.tif']

In [20]:
image = cv2.imread(alignedImages.pop(0), cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_ANYCOLOR)
image = image.astype(np.float64, copy=False)

imageCpt = 0
for i in alignedImages:
    print('Adding {}'.format(i))
    image += cv2.imread(i, cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_ANYCOLOR)
    imageCpt += 1

image = image / imageCpt


Adding /mnt/data/Photos/Repository/tmp/Aligned_0001.tif
Adding /mnt/data/Photos/Repository/tmp/Aligned_0004.tif
Adding /mnt/data/Photos/Repository/tmp/Aligned_0002.tif
Adding /mnt/data/Photos/Repository/tmp/Aligned_0000.tif


In [21]:
image2 = image - image.min()
image2 = (image2 / image2.max()) * 65535.0

In [22]:
image2.max()

65535.0

In [23]:
# Gamma ajustement
image2 = image2.astype(np.float32, copy=False) 
bitDepth = 2.0**16
image2 = np.power(image2/bitDepth, 1/2.1)*bitDepth

In [ ]:
print('Ajust White balance')
image3 = astrolights.histogram.ajustWhiteBalance(image2)
    
#Normalization of the histogram
#print('Normalize the histogram')
#image2 = astrolights.histogram.normalizeHistogram(image2)

In [ ]:
#image2 = astrolights.histogram.normalizeHistogram(image)

In [24]:
image2 = image2.astype(np.uint16, copy=False)

In [25]:
cv2.imwrite(tmpDir + '/' + 'image.tif', image2)

True

In [26]:
# remove files: RAWoffsetFiles, RAWdarkFiles, RAWflatFiles and RAWimageFiles

Make a median from image list:
convert Aligned_* -evaluate-sequence median colors.tif